In [1]:
import torch
import torch.nn as nn
from torchdiffeq import odeint
import sys; sys.path.append(2*'../')
from src import *
import matplotlib.pyplot as plt
from torch.distributions import MultivariateNormal, Uniform
from warnings import warn

# device = torch.device('cuda:0')
device=torch.device('cpu')

## 1. CSTR model

In [2]:
System = ControlledCSTR

##### Scaling since the parameters have very different values
scaling_T_R = 1/100
scaling_T_K = 1/100
scaling_Q_dot = 1/2000
scaling_F = 1/100

# Scale the inputs appropriately for the controller
in_scal = torch.ones(4).to(device)
in_scal[2] = scaling_T_R
in_scal[3] = scaling_T_K
print('Input scaling:\n', in_scal)

out_scal = torch.tensor([[5.,  100.],
                        [-8500, 0.]]).to(device)
print('Output scaling:\n', out_scal)

# State constraints
lower_bounds = [0.1, 0.1, 50., 50.]
upper_bounds = [2., 2., None, 140.]
penalties =  .01*torch.ones(4); penalties[3] = 100
# penalties =  torch.zeros(4)
print('Lower bounds:\n', lower_bounds, '\nUpper bounds:\n', upper_bounds)

Input scaling:
 tensor([1.0000, 1.0000, 0.0100, 0.0100])
Output scaling:
 tensor([[ 5.0000e+00,  1.0000e+02],
        [-8.5000e+03,  0.0000e+00]])
Lower bounds:
 [0.1, 0.1, 50.0, 50.0] 
Upper bounds:
 [2.0, 2.0, None, 140.0]


## 2. Parameters for MPC simulation

In [3]:
# Time constraints
Δt = 0.005
t0, tf = 0, 0.5 # 0.5
t_span = torch.linspace(t0, tf, int(tf/Δt) + 1).to(device) # define the t span

# MPC simulation variables
steps_nom = 10 # Nominal steps to do between each MPC step
max_iters = 50
eps_accept = 1e-3 # so we 'fix' the iterations to be always maximum
lookahead_steps = 10
bs = 512

# Desired final condition
C_b_star = 0.6

# Initial Conditions
ε = .01 # 1% of uncertainty given initial conditions
C_a_0 = 0.8 # This is the initial concentration inside the tank [mol/l]
C_b_0 = 0.5 # This is the controlled variable [mol/l]
T_R_0 = 134.14 #[C]
T_K_0 = 130.0 #[C]
init = torch.Tensor([C_a_0, C_b_0, T_R_0, T_K_0])
init_dist = Uniform((1-ε)*init, (1+ε)*init)
x0 = init_dist.sample((bs,)).to(device)

# Controllers and systems
lr = .5e-3
u = BoxConstrainedController(4, 2, input_scaling=in_scal, output_scaling=out_scal, constrained=True)
const_u = RandConstController([1, 1], -1, 1).to(device) # dummy constant controller for simulation
opt = torch.optim.Adam(u.parameters(), lr=lr) # optimizer
system = System(u, solver='midpoint', retain_u=True)
real_system = System(const_u, solver='dopri5')

## 2b. Define cost function

In [4]:
loss = nn.MSELoss()
class PositioningCost(nn.Module):
    '''Economic version of the positioning cost: we want to
    penalize big control inputs

    Args:
        target: torch.tensor, target position
        Q: float, state weight
        R: float, controller weight
        P: float, terminal cost weight
    '''
    def __init__(self, target, Q=1, R=0, P=0):
        super().__init__()
        self.target = target
        self.Q, self.R, self.P = Q, R, P
        
    def forward(self, traj, u=None, mesh_p=None):
        """
        traj: trajectory to be followed
        u: control input to be minimized
        """
        cost = self.Q*torch.norm(traj[-1, ..., 1] - self.target).mean(0)
        return cost
    
cost_function = PositioningCost(torch.Tensor([C_b_star]))

In [5]:
mpc = TorchMPC(system, cost_function, t_span, opt, eps_accept=eps_accept, max_g_iters=max_iters,
                lookahead_steps=lookahead_steps, lower_bounds=lower_bounds,
                upper_bounds=upper_bounds, penalties=penalties).to(device)

mpc.forward_simulation(real_system, x0, t_span)

with torch.no_grad():
# Save the learned controller and nominal trajectory
    torch.save(mpc.control_inputs, 'data/control_inputs.pt')
    torch.save(mpc.trajectory_nominal, 'data/trajectory.pt')

Starting simulation... Time: 0.0000 s
Inner-loop did not converge, last cost: 0.762 | Time: 0.0050 s
Inner-loop did not converge, last cost: 0.362 | Time: 0.0100 s
Inner-loop did not converge, last cost: 0.372 | Time: 0.0150 s
Inner-loop did not converge, last cost: 0.380 | Time: 0.0200 s
Inner-loop did not converge, last cost: 0.385 | Time: 0.0250 s
Inner-loop did not converge, last cost: 0.387 | Time: 0.0300 s
Inner-loop did not converge, last cost: 0.386 | Time: 0.0350 s
Inner-loop did not converge, last cost: 0.384 | Time: 0.0400 s
Inner-loop did not converge, last cost: 0.381 | Time: 0.0450 s
Inner-loop did not converge, last cost: 0.377 | Time: 0.0500 s
Inner-loop did not converge, last cost: 0.371 | Time: 0.0550 s
Inner-loop did not converge, last cost: 0.366 | Time: 0.0600 s
Inner-loop did not converge, last cost: 0.360 | Time: 0.0650 s
Inner-loop did not converge, last cost: 0.354 | Time: 0.0700 s
Inner-loop did not converge, last cost: 0.347 | Time: 0.0750 s
Inner-loop did no